In [1]:
import os
import time
import re
import json
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from tqdm import tqdm

CHECKPOINT_FILE = "output/checkpoint.json"
OUTPUT_FILE = "output/credit_mutuel_agences_full_progress.csv"

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-gpu")
    # options.add_argument("--headless")  # décommenter pour exécuter en mode sans tête
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1400, 1000)
    driver.set_page_load_timeout(60)
    return driver

def click_if_present(driver, by, selector, timeout=3):
    """Clique sur un élément s’il est présent et cliquable"""
    try:
        elm = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((by, selector))
        )
        elm.click()
        return True
    except Exception:
        return False

def click_button_by_partial_text(driver, tag, partial_text, timeout=3):
    """Clique sur le premier bouton contenant partiellement un texte donné"""
    try:
        elems = WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, tag))
        )
        for e in elems:
            try:
                if partial_text.lower() in e.text.lower():
                    e.click()
                    return True
            except Exception:
                continue
    except Exception:
        pass
    return False

def accept_cookies(driver):
    """Ferme le bandeau de cookies si présent"""
    if click_if_present(driver, By.CSS_SELECTOR, "#cookieLBmainbuttons .ei_mainbuttons .ei_btn_label", timeout=2):
        time.sleep(0.5)
        return
    if click_if_present(driver, By.CSS_SELECTOR, "button[data-accept]", timeout=1):
        time.sleep(0.5)
        return
    for txt in ["Tout accepter", "Accepter", "J’accepte", "OK", "Valider"]:
        if click_button_by_partial_text(driver, "button", txt, timeout=1):
            time.sleep(0.5)
            return
    # aucune pop‑up détectée

def extract_full_agence(html, source_url, region_href, dept_href, loc_href):
    """Extrait toutes les agences depuis une page de localité"""
    soup = BeautifulSoup(html, "html.parser")
    results = []

    # 1. Nouveau format : cartes (li.item) avec titre et corps
    cards = soup.select("li.item")
    for card in cards:
        title_elem = card.select_one(".ei_card_title")
        nom = title_elem.get_text(strip=True) if title_elem else ""
        adresse_text = ""
        telephone = ""
        statut = ""
        body = card.select_one(".ei_card_body")
        if body:
            p_tags = body.find_all("p")
            if p_tags:
                # adresse : 1er paragraphe
                adresse_text = p_tags[0].get_text(" ", strip=True)
                # recherche du téléphone et du statut dans les paragraphes suivants
                for p in p_tags[1:]:
                    tel_anchor = p.find("a", href=re.compile(r"^tel:"))
                    if tel_anchor and not telephone:
                        telephone = tel_anchor.get_text(strip=True)
                    stat_span = p.find("span", class_=re.compile(r"cot[0-9]*"))
                    if stat_span and not statut:
                        statut = stat_span.get_text(strip=True)
        code_postal = ""
        if adresse_text:
            cp_match = re.search(r"\b(\d{5})\b", adresse_text)
            code_postal = cp_match.group(1) if cp_match else ""
        if nom:
            results.append({
                "nom": nom,
                "adresse": adresse_text,
                "code_postal": code_postal,
                "telephone": telephone,
                "statut": statut,
                "region_url": region_href,
                "department_url": dept_href,
                "locality_url": loc_href,
                "source_url": source_url
            })

    # 2. Ancien format (pour compatibilité)
    if not results:
        possible = soup.find(
            lambda tag: tag.name in {"a", "h2", "div"}
            and "CAISSE" in tag.get_text(strip=True).upper()
        )
        if possible:
            nom = possible.get_text(strip=True)
            adresse_parts = []
            for sib in possible.find_next_siblings():
                text = sib.get_text(" ", strip=True)
                if not text:
                    continue
                if "Toutes les localités" in text or "Retour haut de page" in text:
                    break
                adresse_parts.append(text)
                if re.search(r"\b\d{5}\b", text) and len(adresse_parts) >= 1:
                    continue
            adresse_text = ", ".join(adresse_parts)
            cp_match = re.search(r"\b(\d{5})\b", adresse_text)
            code_postal = cp_match.group(1) if cp_match else ""
            tel_elem = soup.find("a", href=re.compile(r"^tel:"))
            telephone = tel_elem.get_text(strip=True) if tel_elem else ""
            statut = ""
            for keyword in ["Ouvert", "Fermé", "Fermée"]:
                if soup.find(string=re.compile(keyword, re.IGNORECASE)):
                    statut = keyword
                    break
            results.append({
                "nom": nom,
                "adresse": adresse_text,
                "code_postal": code_postal,
                "telephone": telephone,
                "statut": statut,
                "region_url": region_href,
                "department_url": dept_href,
                "locality_url": loc_href,
                "source_url": source_url
            })
    return results

def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {"done_localites": []}

def save_checkpoint(cp):
    os.makedirs(os.path.dirname(CHECKPOINT_FILE), exist_ok=True)
    with open(CHECKPOINT_FILE, "w", encoding="utf-8") as f:
        json.dump(cp, f, ensure_ascii=False, indent=2)

def append_to_csv(df):
    os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
    if not os.path.exists(OUTPUT_FILE):
        df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
    else:
        df.to_csv(OUTPUT_FILE, mode="a", header=False, index=False, encoding="utf-8-sig")

def scrape_credit_mutuel_full():
    driver = setup_driver()
    base_url = "https://www.creditmutuel.fr/fr/caisses-et-distributeurs/Regions.aspx"
    driver.get(base_url)
    time.sleep(1)
    accept_cookies(driver)

    checkpoint = load_checkpoint()
    done_localites = set(checkpoint.get("done_localites", []))
    seen = set()

    # Charger les agences déjà récupérées pour éviter les doublons
    if os.path.exists(OUTPUT_FILE):
        existing = pd.read_csv(OUTPUT_FILE, dtype=str)
        for _, row in existing.iterrows():
            key = (row.get("nom", ""), row.get("adresse", ""), row.get("telephone", ""))
            seen.add(key)

    # Récupérer les liens vers les régions
    region_links = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='regionId']"))
    )
    region_hrefs = [a.get_attribute("href") for a in region_links if a.get_attribute("href")]

    for region_href in tqdm(region_hrefs, desc="Régions"):
        try:
            driver.get(region_href)
            time.sleep(1)
            accept_cookies(driver)
        except Exception:
            continue
        # Départements
        dept_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='departmentId']")
        dept_hrefs = [a.get_attribute("href") for a in dept_links if a.get_attribute("href")]
        for dept_href in tqdm(dept_hrefs, desc="  Départements", leave=False):
            try:
                driver.get(dept_href)
                time.sleep(1)
                accept_cookies(driver)
            except Exception:
                continue
            # Localités : liens contenant le paramètre inseeCode
            loc_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='inseeCode']")
            loc_hrefs = [a.get_attribute("href") for a in loc_links if a.get_attribute("href")]
            for loc_href in tqdm(loc_hrefs, desc="    Localités", leave=False):
                if loc_href in done_localites:
                    continue
                try:
                    driver.get(loc_href)
                    time.sleep(1)
                    accept_cookies(driver)
                except Exception as e:
                    print(f"⚠️ Échec chargement {loc_href}: {e}")
                    continue
                html = driver.page_source
                agences = extract_full_agence(html, loc_href, region_href, dept_href, loc_href)
                new_agences = []
                noms = []
                for a in agences:
                    key = (a["nom"], a["adresse"], a["telephone"])
                    if key in seen or not a["nom"]:
                        continue
                    seen.add(key)
                    new_agences.append(a)
                    noms.append(a["nom"])
                if new_agences:
                    df_new = pd.DataFrame(new_agences)
                    append_to_csv(df_new)
                    print(f"✅ {len(new_agences)} agences sauvegardées : {', '.join(noms)}")
                else:
                    print("ℹ️ Pas de nouvelles agences (ou déjà existantes).")
                done_localites.add(loc_href)
                checkpoint["done_localites"] = list(done_localites)
                save_checkpoint(checkpoint)

    driver.quit()
    print("\n✅ Scraping terminé.")

if __name__ == "__main__":
    scrape_credit_mutuel_full()


Régions:   0%|          | 0/12 [00:00<?, ?it/s]



✅ 1 agences sauvegardées : CAISSE DE CREDIT MUTUEL AMBERIEU EN BUGEY


✅ 1 agences sauvegardées : CAISSE DE CREDIT MUTUEL BELLEY


✅ 3 agences sauvegardées : CAISSE DE CREDIT MUTUEL BRESSAN, CAISSE DE CREDIT MUTUEL BRESSAN - BOURG  PROFESSIONNELS, CAISSE DE CREDIT MUTUEL ENSEIGNANT LYON - CME LYON BOURG EN BRESSE


✅ 1 agences sauvegardées : CAISSE DE CREDIT MUTUEL DAGNEUX


...

✅ 1 agences sauvegardées : CAISSE DE CREDIT MUTUEL AGRICULTURE DE ORANGE


✅ 1 agences sauvegardées : CAISSE DE CREDIT MUTUEL VAL DURANCE




Régions: 100%|██████████| 12/12 [2:38:11<00:00, 790.93s/it]

✅ 1 agences sauvegardées : CAISSE DE CREDIT MUTUEL AGRICULTURE DE VALREAS



✅ Scraping terminé.
